In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Read in original (unintegrated), joint, and individual matrices

In [3]:
a_name = 'TCGA-BRCA'
b_name = 'CCLE'

a_path = 'https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/TCGABRCAxCCLE/tcgabrca_pre-jdri.csv'
b_path = 'https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/TCGABRCAxCCLE/ccle_pre-jdri.csv'

a_joint_path = 'https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/TCGABRCAxCCLE/Post-jDRI/tcgabrca-ccle_tcgabrca-joint.csv'
b_joint_path = 'https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/TCGABRCAxCCLE/Post-jDRI/tcgabrca-ccle_ccle-joint.csv'

a_indiv_path = 'https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/TCGABRCAxCCLE/Post-jDRI/tcgabrca-ccle_tcgabrca-individual.csv'
b_indiv_path = 'https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/TCGABRCAxCCLE/Post-jDRI/tcgabrca-ccle_ccle-individual.csv'

a = pd.read_csv(a_path, index_col=0)
b = pd.read_csv(b_path, index_col=0)
a_joint = pd.read_csv(a_joint_path, index_col=0)
b_joint = pd.read_csv(b_joint_path, index_col=0)
a_indiv = pd.read_csv(a_indiv_path, index_col=0)
b_indiv = pd.read_csv(b_indiv_path, index_col=0)

In [3]:
#Read AJIVE Object
import urllib.request
import pickle

ajive = pickle.load(urllib.request.urlopen('https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/TCGABRCAxCCLE/Post-jDRI/tcgabrca-ccle_ajive.p'))

## Figure 2A: Variation Explained Plot

In [8]:
def getVarianceExplained(original, joint, individual, label):
    from numpy.linalg import norm
    joint_var = norm(joint)**2/norm(original)**2
    individual_var = norm(individual)**2/norm(original)**2
    residual_var = 1-joint_var-individual_var
    return pd.DataFrame([np.round(joint_var*100, 2), np.round(individual_var*100,2), np.round(residual_var*100,2)], index=['Joint','Individual','Residual'], columns=[label])

def plotVarianceExplained(df, figsize=[12,8]):
    import matplotlib.patheffects as PathEffects
    sns.set(context='talk', style='ticks')
    sns.set_style('ticks')
    ax = df.plot.bar(stacked=True, figsize=figsize, table=False, rot='horizontal')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels))
    
    for rect in ax.patches:
        # Find where everything is located
        height = rect.get_height()
        width = rect.get_width()
        x = rect.get_x()
        y = rect.get_y()

        # The height of the bar is the data value and can be used as the label
        label_text = f'{height:.2f}'  
        label_x = x + width - 0.16  
        label_y = y + height / 2 
        txt = ax.text(label_x, label_y, label_text + '%', ha='right', va='center', fontsize=16, color='white', fontweight='black')
        txt.set_path_effects([PathEffects.withStroke(linewidth=0.5, foreground='black')])
        
    ax.set_ylabel("% Variation Explained", fontsize=18)
    plt.show()
    
    plt.tight_layout()
       
        
plt_df = getVarianceExplained(a, a_joint, a_indiv, a_name).join(getVarianceExplained(b, b_joint, b_indiv, b_name)).T
plotVarianceExplained(plt_df)

<Figure size 432x288 with 0 Axes>

## Figure 2B

In [9]:
#Unnormalized Scores
a_j_unnorm = ajive.blocks['A'].joint.svals()*ajive.blocks['A'].joint.loadings()
a_i_unnorm = ajive.blocks['A'].individual.svals()*ajive.blocks['A'].individual.loadings()
b_j_unnorm = ajive.blocks['B'].joint.svals()*ajive.blocks['B'].joint.loadings()
b_i_unnorm = ajive.blocks['B'].individual.svals()*ajive.blocks['B'].individual.loadings()

#Proportion of Variation
a_joint_prop = np.sum(a_j_unnorm**2, axis=1)/np.sum(a.T**2, axis=0)
a_indiv_prop = np.sum(a_i_unnorm**2, axis=1)/np.sum(a.T**2, axis=0)
a_prop_df = pd.DataFrame([a_joint_prop, a_indiv_prop], index=['Joint Proportion','Individual Proportion']).T

b_joint_prop = np.sum(b_j_unnorm**2, axis=1)/np.sum(b.T**2, axis=0)
b_indiv_prop = np.sum(b_i_unnorm**2, axis=1)/np.sum(b.T**2, axis=0)

#Remove Duplicate Cell Lines
b_joint_prop = b_joint_prop.drop(b_joint_prop[pd.DataFrame(b_joint_prop).index.duplicated()].index)
b_indiv_prop = b_indiv_prop.drop(b_indiv_prop[pd.DataFrame(b_indiv_prop).index.duplicated()].index)
b_prop_df = pd.DataFrame([b_joint_prop, b_indiv_prop], index=['Joint Proportion','Individual Proportion']).T

#Determine Tissue from cell line ID
b_prop_df['TISSUE'] = b_prop_df.index.map(lambda i: '_'.join(i.split('_')[1:]))

#Calculate Ratio Metric
b_prop_df['%Joint'] = b_prop_df['Joint Proportion']*100
b_prop_df['%Individual'] = b_prop_df['Individual Proportion']*100
b_prop_df['Ratio'] = b_prop_df.apply(lambda i: i['Joint Proportion']/i['Individual Proportion'], axis=1)

#Plot
tissue_sort = b_prop_df.reset_index().groupby('TISSUE').mean().sort_values('Ratio').index.tolist()
plt.figure(figsize=[12,8], facecolor='w')
sns.boxplot(data=b_prop_df, x='TISSUE', y='Ratio', order=tissue_sort)
plt.xticks(size=12, rotation=55, ha='right', rotation_mode="anchor");
plt.title('Cell Line Tissue Population Best Represented by TCGA-BRCA')
plt.ylabel(r'$\frac{Joint\ Proportion}{Individual\ Proportion}$')


Text(0, 0.5, '$\\frac{Joint\\ Proportion}{Individual\\ Proportion}$')

# UMAP 

#### Example of applying UMAP to unintegrated datasets

In [27]:
from umap import UMAP

umap = UMAP(n_neighbors=100, n_epochs=1000, metric='cosine', verbose=True)

umap_x = np.concatenate([np.array(a), np.array(b)])
umap_x_fit = umap.fit_transform(X)

UMAP(angular_rp_forest=True, metric='cosine', n_epochs=1000, n_neighbors=100, verbose=True)
Tue Aug  9 19:29:05 2022 Construct fuzzy simplicial set
Tue Aug  9 19:30:57 2022 Finding Nearest Neighbors
Tue Aug  9 19:30:57 2022 Finished Nearest Neighbor Search
Tue Aug  9 19:30:57 2022 Construct embedding


Epochs completed:   0%|            0/1000 [00:00]

Tue Aug  9 19:31:04 2022 Finished embedding


In [28]:
plt_df = pd.DataFrame(umap_x_fit, index=a.index.tolist()+b.index.tolist(), columns=['UMAP1','UMAP2'])

plt.figure(figsize=(15,10))
sns.set_context('poster')
sns.scatterplot(x=umap_x_fit[a.shape[0]:,0], 
                y=umap_x_fit[a.shape[0]:,1],
                label='Cell Lines', marker='X', color='grey', alpha=0.7)
sns.scatterplot(x=umap_x_fit[:a.shape[0],0], 
                y=umap_x_fit[:a.shape[0],1],
                label='Tumors', alpha=0.9, color='hotpink')

plt.xticks(fontsize=12)
plt.xlabel('UMAP direction 1', fontsize=20, color='black')
plt.ylabel('UMAP direction 2', fontsize=20, color='black')
plt.legend(fontsize=20, 
           ncol=1,
           bbox_to_anchor=(1.25, 1.))
plt.tight_layout()
plt.show()

# Joint Statistic

In [10]:
def distance(x1, y1, a=-1, b=1, c=0):  
    return abs((a * x1 + b * y1 + c)) / (np.sqrt(a * a + b * b)) 

def joint_statistics(joint, individual):
    s = 0.05
    return np.var(joint, axis=1).clip(lower=s)-(np.var(individual, axis=1).clip(lower=s))

def permute_expected_statistics(joint, individual, n_perm=20):
    joined = joint.join(individual, lsuffix='_J', rsuffix='_I')
    perms = []
    sig_counts = []
    for p in range(n_perm):
        col_perm = np.random.permutation(joined.columns.tolist())
        j_perm = joined[col_perm[:len(col_perm)//2]]
        i_perm = joined[col_perm[len(col_perm)//2:]]
        exp_stat = np.sort(joint_statistics(j_perm, i_perm))[::-1]
        perms.append(exp_stat)
    
    return pd.DataFrame(perms)

In [11]:
#Calculate observed and expected joint statistics
a_obs_j = pd.Series(joint_statistics(a_joint, a_indiv))
a_expect_j_perms = permute_expected_statistics(a_joint, a_indiv, n_perm=200)
a_expect_j = pd.DataFrame(a_expect_j_perms).mean()
a_expect_j.index = a_joint.index
a_obs_j_sort = a_obs_j.sort_values(ascending=False)
a_stat_df = pd.DataFrame([np.sort(a_obs_j)[::-1], a_expect_j], index=['Observed','Expected'], columns=a_obs_j_sort.index).T

b_obs_j = pd.Series(joint_statistics(b_joint, b_indiv))
b_obs_j_sort = b_obs_j.sort_values(ascending=False)
b_expect_j_perms = permute_expected_statistics(b_joint, b_indiv, n_perm=200)
b_expect_j = pd.DataFrame(b_expect_j_perms).mean()
b_expect_j.index = b_joint.index
b_stat_df = pd.DataFrame([np.sort(b_obs_j)[::-1], b_expect_j], index=['Observed','Expected'], columns=b_obs_j_sort.index).T

#Combine into DataFrame
jstats_df = a_stat_df.join(b_stat_df, lsuffix='_'+str(a_name), rsuffix='_'+str(b_name))

In [12]:
jstats_df.head()

Observed_TCGA-BRCA  Expected_TCGA-BRCA  Observed_CCLE  \
AL589842.1              4.393258            0.498305       2.126029   
VGLL1                   3.478945            0.399893       5.472110   
TFF3                    3.362353            0.358533       4.687231   
KRT16                   3.064560            0.333747       5.538616   
RP11-63M22.2            3.011258            0.316496       0.852571   

              Expected_CCLE  
AL589842.1         0.091508  
VGLL1              0.331062  
TFF3               0.266400  
KRT16              0.340536  
RP11-63M22.2       0.023613

In [14]:
import matplotlib.patches as mpatches

sns.set_context('talk')
plt.figure(figsize=[14,10])
sns.set_context('talk')
sns.set_palette('deep')

ax = sns.scatterplot(data=jstats_df, x='Observed_TCGA-BRCA', y='Observed_CCLE', alpha=0.3, color='grey')
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]
ax.plot(lims, lims, '--', alpha=0.75, zorder=0)
plt.axvline(0, color='k', linestyle='-')
plt.axhline(0, color='k', linestyle='-')
plt.xlabel('Observed TCGABRCA Joint Statistic')
plt.ylabel('Observed CCLE Joint Statistic')

Text(0, 0.5, 'Observed CCLE Joint Statistic')

# Elastic Net Modeling

#### Trains elastic net models using an exhaustive grid search
#### Final model was selected using best mean $R^2$ across iterations

In [50]:
from sklearn.linear_model import ElasticNet, SGDClassifier, enet_path
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from scipy.stats import linregress


def rsquared(x, y):  
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return r_value**2

def generate_alphas(X, y, l1_ratios, n_alphas):
    alpha_paths = []
    for l1r in l1_ratios:
        alpha_paths.append(enet_path(X, y, l1r, n_alphas=n_alphas))
    return [l1_ratios, alpha_paths]

#Linear Elastic Net Regression
def enet_modeling(X, y, n_alphas=100, n_splits=20, test_size=0.33, l1_ratios=[0.1, 0.5, 0.7, 0.9, 0.95, 1]):    
    params = generate_alphas(X, y, l1_ratios=l1_ratios, n_alphas=n_alphas)
    model = ElasticNet()
    parameters = {'l1_ratio':params[0], 'alpha':[i[0] for i in params[1]]}
    my_scorer = make_scorer(rsquared, greater_is_better=True)
    splits = ShuffleSplit(n_splits=n_splits, test_size=test_size)
    custom_cv = [(train,test) for (train,test) in splits.split(X)]
    
    param_dict = []
    i = 0
    for i in range(6):
        for j in range(100):
            param_dict.append({'l1_ratio':[params[0][i]], 'alpha':[[x[0] for x in params[1]][i][j]]})

    gscv = GridSearchCV(X=StandardScaler().fit_transform(X[genelist]), estimator=model, param_grid=param_dict, scoring=my_scorer, cv=custom_cv, n_jobs=-1)
    return gscv

#Logistic Elastic Net Regression
def enet_modeling_log(X, y, n_alphas=100, n_splits=20, test_size=0.33, l1_ratios=[0.1, 0.5, 0.7, 0.9, 0.95, 1]):
    params = generate_alphas(X, y, l1_ratios=l1_ratios, n_alphas=n_alphas)
    model = SGDClassifier(loss='log', penalty='elasticnet')
    parameters = {'l1_ratio':params[0], 'alpha':[i[0] for i in params[1]]}
    splits = ShuffleSplit(n_splits=n_splits, test_size=test_size)
    custom_cv = [(train,test) for (train,test) in splits.split(X)]
    
    param_dict = []
    i = 0
    for i in range(6):
        for j in range(100):
            param_dict.append({'l1_ratio':[params[0][i]], 'alpha':[[x[0] for x in params[1]][i][j]]})

    gscv = GridSearchCV(X=StandardScaler().fit_transform(X[genelist]), estimator=model, param_grid=param_dict, scoring='f1', cv=custom_cv, n_jobs=-1)
    return gscv

### Cell line drug response data downloaded from CTRP v2

https://ctd2-data.nci.nih.gov/Public/Broad/CTRPv2.1_2016_pub_NatChemBiol_12_109/

Example lapatinib response data has been copied and uploaded from this public dataset

In [30]:
drug_path = 'https://webshare.bioinf.unc.edu/public/baprice/AJIVE_jDR_Integration/DrugData/ccle_lapatinib_auc.csv'
lapatinib = pd.read_csv(drug_path, index_col=0)

In [46]:
df = b.join(lapatinib).dropna()
X = df[b.columns]
y = df['LAPATINIB']

In [47]:
X.head(3)

DDX11L1    WASH7P  RP11-34P13.3   FAM138A  \
22RV1_PROSTATE                 0.575571  0.149130     -0.327859 -0.334919   
253JBV_URINARY_TRACT           0.243810  0.506233      0.647678  3.039768   
42MGBA_CENTRAL_NERVOUS_SYSTEM -0.775893  0.511025      0.245783 -0.334919   

                                 OR4G4P   OR4G11P     OR4F5  RP11-34P13.7  \
22RV1_PROSTATE                -0.069974 -0.022882 -0.025035      0.847763   
253JBV_URINARY_TRACT           0.980002 -0.022882  0.456894     -0.510602   
42MGBA_CENTRAL_NERVOUS_SYSTEM -0.069974 -0.022882 -0.025035     -0.362404   

                               RP11-34P13.8    CICP27  ...    MT-CO2  \
22RV1_PROSTATE                     0.183748  0.907296  ... -0.282142   
253JBV_URINARY_TRACT               0.047202 -0.367859  ... -0.076084   
42MGBA_CENTRAL_NERVOUS_SYSTEM      0.206120 -0.676622  ... -0.334194   

                                MT-ATP8   MT-ATP6    MT-CO3    MT-ND3  \
22RV1_PROSTATE                -2.083604 -0.273867 -0.126954 -0.857323   
253JBV_URINARY_TRACT           0.019612 -0.109078 -0.110159  0.175195   
42MGBA_CENTRAL_NERVOUS_SYSTEM -0.082449 -0.200066 -0.763888 -0.732326   

                                MT-ND4L    MT-ND4    MT-ND5    MT-ND6  \
22RV1_PROSTATE                -0.925390 -0.256161 -0.024146 -0.567404   
253JBV_URINARY_TRACT          -0.130250 -0.147294 -0.051593 -0.145113   
42MGBA_CENTRAL_NERVOUS_SYSTEM -0.485535 -0.535861 -0.463347 -0.220031   

                                 MT-CYB  
22RV1_PROSTATE                 0.167963  
253JBV_URINARY_TRACT          -0.600159  
42MGBA_CENTRAL_NERVOUS_SYSTEM -0.322003  

[3 rows x 49843 columns]

In [48]:
y.head(3)

22RV1_PROSTATE                   13.476
253JBV_URINARY_TRACT             13.244
42MGBA_CENTRAL_NERVOUS_SYSTEM    13.782
Name: LAPATINIB, dtype: float64

In [ ]:
gscv = enet_modeling(X, y)